In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

---
## Récupération des jeux de données et compréhension

In [91]:
df_matches = pd.read_csv("Africa Cup of Nations Matches MaJ.csv", sep=";")
df_countries = pd.read_csv("African Countries.csv", sep=";")
df_stats = pd.read_csv("Tournaments General Statistics MaJ.csv", sep=";")

In [92]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  674 non-null    int64  
 1   Date                  590 non-null    object 
 2   Time                  333 non-null    object 
 3   HomeTeam              674 non-null    object 
 4   AwayTeam              674 non-null    object 
 5   HomeTeamGoals         670 non-null    float64
 6   AwayTeamGoals         670 non-null    float64
 7   Stage                 674 non-null    object 
 8   SpecialWinConditions  63 non-null     object 
 9   Stadium               670 non-null    object 
 10  City                  670 non-null    object 
 11  Attendance            522 non-null    float64
dtypes: float64(3), int64(1), object(8)
memory usage: 63.3+ KB


In [93]:
df_matches.isna().sum()

Year                      0
Date                     84
Time                    341
HomeTeam                  0
AwayTeam                  0
HomeTeamGoals             4
AwayTeamGoals             4
Stage                     0
SpecialWinConditions    611
Stadium                   4
City                      4
Attendance              152
dtype: int64

In [94]:
df_matches.loc[df_matches['HomeTeamGoals'].isna()]

,Year,Date,Time,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,Stage,SpecialWinConditions,Stadium,City,Attendance
1,1957,10/02/1957,NaN,Ethiopia,South Africa,NaN,NaN,Semifinals,Ethiopia wins due to disqualification of othe...,NaN,NaN,NaN
489,2010,11/01/2010,19:30,Ghana,Togo,NaN,NaN,Group B,Ghana wins due to disqualification of other team,NaN,NaN,NaN
490,2010,15/01/2010,17:00,Burkina Faso,Togo,NaN,NaN,Group B,Burkina Faso wins due to disqualification of ...,NaN,NaN,NaN
493,2010,19/01/2010,17:00,Ivory Coast,Togo,NaN,NaN,Group B,Ivory Coast wins due to disqualification of o...,NaN,NaN,NaN


Nous allons supprimer tous les matchs qui n'ont pas été joués

In [95]:
df_matches.dropna(subset=['HomeTeamGoals'], inplace=True)

In [96]:
df_matches.isna().sum()

Year                      0
Date                     84
Time                    340
HomeTeam                  0
AwayTeam                  0
HomeTeamGoals             0
AwayTeamGoals             0
Stage                     0
SpecialWinConditions    611
Stadium                   0
City                      0
Attendance              148
dtype: int64

In [97]:
# Conversion des types float en int sur les buts et le nombre de personnes au stade
df_matches['HomeTeamGoals'] = df_matches['HomeTeamGoals'].astype(int)
df_matches['AwayTeamGoals'] = df_matches['AwayTeamGoals'].astype(int)

In [98]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 670 entries, 0 to 673
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  670 non-null    int64  
 1   Date                  586 non-null    object 
 2   Time                  330 non-null    object 
 3   HomeTeam              670 non-null    object 
 4   AwayTeam              670 non-null    object 
 5   HomeTeamGoals         670 non-null    int32  
 6   AwayTeamGoals         670 non-null    int32  
 7   Stage                 670 non-null    object 
 8   SpecialWinConditions  59 non-null     object 
 9   Stadium               670 non-null    object 
 10  City                  670 non-null    object 
 11  Attendance            522 non-null    float64
dtypes: float64(1), int32(2), int64(1), object(8)
memory usage: 62.8+ KB


In [99]:
df_matches['HomeTeam'].nunique()

64

In [100]:
df_matches['HomeTeam'] = df_matches['HomeTeam'].str.strip()

In [101]:
df_matches['HomeTeam'].nunique()

44

In [102]:
df_matches['AwayTeam'] = df_matches['AwayTeam'].str.strip()

---
## Feature Engineering pour déterminer si l'équipe a gagné (ou perdu) à domicile (ou à l'extérieur) et déterminer le gagnant du match

In [103]:
df_matches['HomeWin'] = df_matches['HomeTeamGoals'] > df_matches['AwayTeamGoals']
df_matches['AwayWin'] = df_matches['AwayTeamGoals'] > df_matches['HomeTeamGoals']

df_matches['HomeLoss'] = df_matches['AwayWin']
df_matches['AwayLoss'] = df_matches['HomeWin']

df_matches['Draw'] = df_matches['HomeTeamGoals'] == df_matches['AwayTeamGoals']

In [104]:
df_matches.sample(5)

,Year,Date,Time,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,Stage,SpecialWinConditions,Stadium,City,Attendance,HomeWin,AwayWin,HomeLoss,AwayLoss,Draw
564,2017,NaN,17:00,DR Congo,Ghana,1,2,Quarter-finals,NaN,Stade d'Oyem,Oyem,NaN,False,True,True,False,False
615,2019,NaN,21:00,Nigeria,South Africa,2,1,Quarter-finals,NaN,Cairo International Stadium,Cairo,48343.0,True,False,False,True,False
100,1976,04/03/1976,NaN,Morocco,Zaire,1,0,Group B,NaN,Dire Dawa Stadium,Dire Dawa,NaN,True,False,False,True,False
631,2022,12/01/2022,NaN,Equatorial Guinea,Ivory Coast,0,1,Group E,NaN,Japoma Stadium,Douala,NaN,False,True,True,False,False
185,1986,17/03/1986,NaN,Cameroon,Ivory Coast,1,0,Semifinals,NaN,Alexandria Stadium,Alexandria,12000.0,True,False,False,True,False


In [105]:
df_matches['Winner'] = np.where(~df_matches['Draw'],
                                np.where(df_matches['HomeWin'], df_matches['HomeTeam'], df_matches['AwayTeam']),
                                df_matches['SpecialWinConditions'].apply(lambda x: x.split(' win')[0] if isinstance(x, str) else ''))

---
# Calcul du total de victoires, défaites et matchs nuls par équipe.

In [106]:
total_wins = pd.concat([df_matches.groupby('HomeTeam')['HomeWin'].sum(),
                        df_matches.groupby('AwayTeam')['AwayWin'].sum()], axis=1, sort=False).sum(axis=1)

In [107]:
total_losses = pd.concat([df_matches.groupby('HomeTeam')['HomeLoss'].sum(),
                        df_matches.groupby('AwayTeam')['AwayLoss'].sum()], axis=1, sort=False).sum(axis=1)

In [120]:
total_draws = pd.concat([df_matches.groupby('HomeTeam')['Draw'].sum(),
                        df_matches.groupby('AwayTeam')['Draw'].sum()], axis=1, sort=False).sum(axis=1)

Algeria                21.0
Angola                 10.0
Benin                   5.0
Burkina Faso           13.0
Burundi                 0.0
Cameroon               28.0
Cape Verde              1.0
Comoros                 0.0
Congo                   6.0
Congo-Kinshasa          1.0
DR Congo                8.0
Egypt                  19.0
Equatorial Guinea       2.0
Ethiopia                3.0
Gabon                  10.0
Gambia                  1.0
Ghana                  19.0
Guinea                 13.0
Guinea-Bissau           3.0
Ivory Coast            24.0
Kenya                   4.0
Liberia                 2.0
Libya                   4.0
Madagascar              2.0
Malawi                  2.0
Mali                   15.0
Mauritania              2.0
Morocco                21.0
Mozambique              2.0
Namibia                 2.0
Nigeria                19.0
Rwanda                  1.0
Senegal                16.0
Sierra Leone            3.0
South Africa           10.0
Sudan               

In [123]:
total_games = pd.concat([df_matches['HomeTeam'].value_counts(),
                         df_matches['AwayTeam'].value_counts()], axis=1, sort=False).sum(axis=1)
total_games

Cameroon                88.0
Ghana                   93.0
Egypt                  102.0
Nigeria                 94.0
Ivory Coast             90.0
Morocco                 64.0
Algeria                 71.0
Senegal                 61.0
Zambia                  58.0
Tunisia                 71.0
South Africa            37.0
Burkina Faso            36.0
Mali                    45.0
DR Congo                31.0
Guinea                  41.0
Ethiopia                27.0
Gabon                   19.0
Congo                   20.0
Zaire                   26.0
Angola                  20.0
Sudan                   24.0
Uganda                  23.0
Libya                    8.0
Togo                    22.0
United Arab Rep.         5.0
Zimbabwe                15.0
Malawi                  10.0
Guinea-Bissau            9.0
Congo-Kinshasa           8.0
Madagascar               5.0
Mauritania               6.0
Kenya                   17.0
Benin                   14.0
Namibia                  9.0
Rwanda        

In [124]:
# Transformer ces séries en dictionnaire
dict_games = total_games.to_dict()
dict_wins = total_wins.to_dict()
dict_losses = total_losses.to_dict()
dict_draws = total_draws.to_dict()

In [125]:
dict_games

{'Cameroon': 88.0,
 'Ghana': 93.0,
 'Egypt': 102.0,
 'Nigeria': 94.0,
 'Ivory Coast': 90.0,
 'Morocco': 64.0,
 'Algeria': 71.0,
 'Senegal': 61.0,
 'Zambia': 58.0,
 'Tunisia': 71.0,
 'South Africa': 37.0,
 'Burkina Faso': 36.0,
 'Mali': 45.0,
 'DR Congo': 31.0,
 'Guinea': 41.0,
 'Ethiopia': 27.0,
 'Gabon': 19.0,
 'Congo': 20.0,
 'Zaire': 26.0,
 'Angola': 20.0,
 'Sudan': 24.0,
 'Uganda': 23.0,
 'Libya': 8.0,
 'Togo': 22.0,
 'United Arab Rep.': 5.0,
 'Zimbabwe': 15.0,
 'Malawi': 10.0,
 'Guinea-Bissau': 9.0,
 'Congo-Kinshasa': 8.0,
 'Madagascar': 5.0,
 'Mauritania': 6.0,
 'Kenya': 17.0,
 'Benin': 14.0,
 'Namibia': 9.0,
 'Rwanda': 3.0,
 'Liberia': 5.0,
 'Sierra Leone': 8.0,
 'Gambia': 5.0,
 'Mozambique': 12.0,
 'Burundi': 3.0,
 'Tanzania': 6.0,
 'Comoros': 4.0,
 'Equatorial Guinea': 9.0,
 'Cape Verde': 5.0,
 'Upper Volta': 3.0,
 'Mauritius': 3.0,
 'Congo-Brazzaville': 3.0,
 'Congo-Léopoldville': 2.0}

In [126]:
# Constitution d'une série avec les équipes et les éditions auxquels ils ont participé...
unique_home_teams_and_years = df_matches[["HomeTeam", "Year"]].drop_duplicates()
unique_home_teams_and_years.rename(columns={'HomeTeam': 'Team'}, inplace=True)

unique_away_teams_and_years = df_matches[["AwayTeam", "Year"]].drop_duplicates()
unique_away_teams_and_years.rename(columns={'AwayTeam': 'Team'}, inplace=True)

unique_teams_and_years = (pd.concat([unique_home_teams_and_years, unique_away_teams_and_years])).drop_duplicates()
unique_teams_and_years

,Team,Year
0,Sudan,1957
2,Egypt,1957
3,Egypt,1959
4,Sudan,1959
6,Ethiopia,1962
...,...,...
522,Cape Verde,2013
525,Togo,2013
531,Equatorial Guinea,2015
532,Guinea,2015


In [127]:
# ... Calcul du nombre  et transformation en dictionnaire 
dict_tournaments = unique_teams_and_years['Team'].value_counts().to_dict()
dict_tournaments

{'Egypt': 24,
 'Ivory Coast': 24,
 'Ghana': 23,
 'Cameroon': 19,
 'Nigeria': 19,
 'Tunisia': 19,
 'Algeria': 18,
 'Morocco': 16,
 'Zambia': 15,
 'Senegal': 14,
 'Guinea': 12,
 'Mali': 11,
 'Ethiopia': 10,
 'South Africa': 9,
 'Burkina Faso': 9,
 'Sudan': 9,
 'DR Congo': 8,
 'Togo': 8,
 'Gabon': 7,
 'Zaire': 7,
 'Uganda': 7,
 'Angola': 6,
 'Kenya': 6,
 'Congo': 6,
 'Zimbabwe': 5,
 'Mozambique': 4,
 'Benin': 4,
 'Guinea-Bissau': 3,
 'Equatorial Guinea': 3,
 'Namibia': 3,
 'Sierra Leone': 3,
 'Malawi': 3,
 'Libya': 2,
 'Tanzania': 2,
 'Mauritania': 2,
 'Liberia': 2,
 'Congo-Kinshasa': 2,
 'Cape Verde': 2,
 'Rwanda': 1,
 'Gambia': 1,
 'Mauritius': 1,
 'Congo-Brazzaville': 1,
 'Congo-Léopoldville': 1,
 'United Arab Rep.': 1,
 'Comoros': 1,
 'Burundi': 1,
 'Madagascar': 1,
 'Upper Volta': 1}

In [128]:
total_goals_scored = pd.concat([df_matches.groupby('HomeTeam')['HomeTeamGoals'].sum(),
                        df_matches.groupby('AwayTeam')['AwayTeamGoals'].sum()], axis=1, sort=False).sum(axis=1)

In [129]:
dict_goals_scored = total_goals_scored.to_dict()
dict_goals_scored

{'Algeria': 87.0,
 'Angola': 25.0,
 'Benin': 7.0,
 'Burkina Faso': 37.0,
 'Burundi': 0.0,
 'Cameroon': 135.0,
 'Cape Verde': 2.0,
 'Comoros': 4.0,
 'Congo': 21.0,
 'Congo-Kinshasa': 12.0,
 'DR Congo': 35.0,
 'Egypt': 158.0,
 'Equatorial Guinea': 5.0,
 'Ethiopia': 30.0,
 'Gabon': 16.0,
 'Gambia': 4.0,
 'Ghana': 119.0,
 'Guinea': 51.0,
 'Guinea-Bissau': 2.0,
 'Ivory Coast': 129.0,
 'Kenya': 11.0,
 'Liberia': 5.0,
 'Libya': 8.0,
 'Madagascar': 7.0,
 'Malawi': 9.0,
 'Mali': 57.0,
 'Mauritania': 1.0,
 'Morocco': 75.0,
 'Mozambique': 4.0,
 'Namibia': 10.0,
 'Nigeria': 133.0,
 'Rwanda': 3.0,
 'Senegal': 72.0,
 'Sierra Leone': 4.0,
 'South Africa': 41.0,
 'Sudan': 25.0,
 'Tanzania': 5.0,
 'Togo': 15.0,
 'Tunisia': 99.0,
 'Uganda': 21.0,
 'United Arab Rep.': 10.0,
 'Zaire': 34.0,
 'Zambia': 72.0,
 'Zimbabwe': 16.0,
 'Congo-Brazzaville': 2.0,
 'Congo-Léopoldville': 2.0,
 'Mauritius': 2.0,
 'Upper Volta': 2.0}

In [130]:
total_goals_conceded = pd.concat([df_matches.groupby('HomeTeam')['AwayTeamGoals'].sum(),
                                df_matches.groupby('AwayTeam')['HomeTeamGoals'].sum()], axis=1, sort=False).sum(axis=1)
dict_goals_conceded = total_goals_conceded.to_dict()
dict_goals_conceded

{'Algeria': 82.0,
 'Angola': 30.0,
 'Benin': 24.0,
 'Burkina Faso': 52.0,
 'Burundi': 4.0,
 'Cameroon': 80.0,
 'Cape Verde': 6.0,
 'Comoros': 7.0,
 'Congo': 30.0,
 'Congo-Kinshasa': 10.0,
 'DR Congo': 42.0,
 'Egypt': 85.0,
 'Equatorial Guinea': 11.0,
 'Ethiopia': 60.0,
 'Gabon': 25.0,
 'Gambia': 3.0,
 'Ghana': 81.0,
 'Guinea': 60.0,
 'Guinea-Bissau': 12.0,
 'Ivory Coast': 98.0,
 'Kenya': 31.0,
 'Liberia': 7.0,
 'Libya': 9.0,
 'Madagascar': 7.0,
 'Malawi': 15.0,
 'Mali': 57.0,
 'Mauritania': 11.0,
 'Morocco': 55.0,
 'Mozambique': 26.0,
 'Namibia': 24.0,
 'Nigeria': 100.0,
 'Rwanda': 3.0,
 'Senegal': 46.0,
 'Sierra Leone': 14.0,
 'South Africa': 37.0,
 'Sudan': 38.0,
 'Tanzania': 14.0,
 'Togo': 39.0,
 'Tunisia': 84.0,
 'Uganda': 38.0,
 'United Arab Rep.': 5.0,
 'Zaire': 37.0,
 'Zambia': 61.0,
 'Zimbabwe': 31.0,
 'Congo-Brazzaville': 8.0,
 'Congo-Léopoldville': 8.0,
 'Mauritius': 8.0,
 'Upper Volta': 9.0}

In [131]:
df_summary = pd.DataFrame({'nb_tournaments' : dict_tournaments,
                          'nb_games' : dict_games,
                          'nb_wins' : dict_wins,
                          'nb_losses' : dict_losses,
                          'nb_draws' : dict_draws,
                          'nb_goals_scored' : dict_goals_scored,
                          'nb_goals_conceded' : dict_goals_conceded}
                          )

df_summary

,nb_tournaments,nb_games,nb_wins,nb_losses,nb_draws,nb_goals_scored,nb_goals_conceded
Egypt,24,102.0,57.0,26.0,19.0,158.0,85.0
Ivory Coast,24,90.0,38.0,28.0,24.0,129.0,98.0
Ghana,23,93.0,48.0,26.0,19.0,119.0,81.0
Cameroon,19,88.0,45.0,15.0,28.0,135.0,80.0
Nigeria,19,94.0,52.0,23.0,19.0,133.0,100.0
Tunisia,19,71.0,22.0,24.0,25.0,99.0,84.0
Algeria,18,71.0,26.0,24.0,21.0,87.0,82.0
Morocco,16,64.0,26.0,17.0,21.0,75.0,55.0
Zambia,15,58.0,24.0,20.0,14.0,72.0,61.0
Senegal,14,61.0,26.0,19.0,16.0,72.0,46.0


In [132]:
df_summary.reset_index(inplace=True)
df_summary

,index,nb_tournaments,nb_games,nb_wins,nb_losses,nb_draws,nb_goals_scored,nb_goals_conceded
0,Egypt,24,102.0,57.0,26.0,19.0,158.0,85.0
1,Ivory Coast,24,90.0,38.0,28.0,24.0,129.0,98.0
2,Ghana,23,93.0,48.0,26.0,19.0,119.0,81.0
3,Cameroon,19,88.0,45.0,15.0,28.0,135.0,80.0
4,Nigeria,19,94.0,52.0,23.0,19.0,133.0,100.0
5,Tunisia,19,71.0,22.0,24.0,25.0,99.0,84.0
6,Algeria,18,71.0,26.0,24.0,21.0,87.0,82.0
7,Morocco,16,64.0,26.0,17.0,21.0,75.0,55.0
8,Zambia,15,58.0,24.0,20.0,14.0,72.0,61.0
9,Senegal,14,61.0,26.0,19.0,16.0,72.0,46.0


In [133]:
df_summary.rename(columns={'index' : 'team'}, inplace=True)
df_summary

,Team,nb_tournaments,nb_games,nb_wins,nb_losses,nb_draws,nb_goals_scored,nb_goals_conceded
0,Egypt,24,102.0,57.0,26.0,19.0,158.0,85.0
1,Ivory Coast,24,90.0,38.0,28.0,24.0,129.0,98.0
2,Ghana,23,93.0,48.0,26.0,19.0,119.0,81.0
3,Cameroon,19,88.0,45.0,15.0,28.0,135.0,80.0
4,Nigeria,19,94.0,52.0,23.0,19.0,133.0,100.0
5,Tunisia,19,71.0,22.0,24.0,25.0,99.0,84.0
6,Algeria,18,71.0,26.0,24.0,21.0,87.0,82.0
7,Morocco,16,64.0,26.0,17.0,21.0,75.0,55.0
8,Zambia,15,58.0,24.0,20.0,14.0,72.0,61.0
9,Senegal,14,61.0,26.0,19.0,16.0,72.0,46.0


In [135]:
df_summary['win_percentage'] = (df_summary['nb_wins']/df_summary['nb_games'])*100
df_summary['loss_percentage'] = (df_summary['nb_losses']/df_summary['nb_games'])*100
df_summary['draw_percentage'] = (df_summary['nb_draws']/df_summary['nb_games'])*100

In [136]:
df_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Team               48 non-null     object 
 1   nb_tournaments     48 non-null     int64  
 2   nb_games           48 non-null     float64
 3   nb_wins            48 non-null     float64
 4   nb_losses          48 non-null     float64
 5   nb_draws           48 non-null     float64
 6   nb_goals_scored    48 non-null     float64
 7   nb_goals_conceded  48 non-null     float64
 8   win_percentage     48 non-null     float64
 9   loss_percentage    48 non-null     float64
 10  draw_percentage    48 non-null     float64
dtypes: float64(9), int64(1), object(1)
memory usage: 4.2+ KB


In [137]:
df_summary.describe()

,nb_tournaments,nb_games,nb_wins,nb_losses,nb_draws,nb_goals_scored,nb_goals_conceded,win_percentage,loss_percentage,draw_percentage
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,7.416667,27.916667,10.208333,10.208333,7.500000,33.833333,33.833333,24.918267,50.534157,24.547576
std,6.921546,29.436432,15.195126,7.863079,7.938594,42.783141,28.218248,18.562869,24.639915,13.286166
min,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,3.000000,0.000000,17.045455,0.000000
25%,2.000000,5.750000,1.000000,3.000000,1.750000,4.000000,9.000000,11.096257,31.471096,18.137255
50%,5.500000,16.000000,3.000000,9.000000,3.500000,13.500000,28.000000,25.000000,43.055556,25.403226
75%,10.250000,38.000000,12.750000,15.250000,13.000000,43.500000,52.750000,40.135135,66.666667,33.333333
max,24.000000,102.000000,57.000000,28.000000,28.000000,158.000000,100.000000,60.000000,100.000000,52.631579


In [138]:
df_summary['nb_tournaments'] = df_summary['nb_tournaments'].astype(int)
df_summary['nb_games'] = df_summary['nb_games'].astype(int)
df_summary['nb_wins'] = df_summary['nb_wins'].astype(int)
df_summary['nb_draws'] = df_summary['nb_draws'].astype(int)
df_summary['nb_losses'] = df_summary['nb_losses'].astype(int)
df_summary['nb_goals_scored'] = df_summary['nb_goals_scored'].astype(int)
df_summary['nb_goals_conceded'] = df_summary['nb_goals_conceded'].astype(int)

In [139]:
df_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Team               48 non-null     object 
 1   nb_tournaments     48 non-null     int32  
 2   nb_games           48 non-null     int32  
 3   nb_wins            48 non-null     int32  
 4   nb_losses          48 non-null     int32  
 5   nb_draws           48 non-null     int32  
 6   nb_goals_scored    48 non-null     int32  
 7   nb_goals_conceded  48 non-null     int32  
 8   win_percentage     48 non-null     float64
 9   loss_percentage    48 non-null     float64
 10  draw_percentage    48 non-null     float64
dtypes: float64(3), int32(7), object(1)
memory usage: 2.9+ KB
